In [2]:
import vertexai
from vertexai.generative_models import GenerativeModel
from google.cloud import storage
import io,json 

from PIL import Image as PIL_Image

PROJECT_ID = ""
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

bucket_name = ""
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

def load_image_from_gcs(bucket_name: str, blob_name: str) -> 'Image':
    """Load an image from Google Cloud Storage into the format expected by Vertex AI."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    image_bytes = blob.download_as_bytes()
    
    pil_image = PIL_Image.open(io.BytesIO(image_bytes))
    pil_image_rgb = pil_image.convert("RGB")
    
    img_byte_arr = io.BytesIO()
    pil_image_rgb.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    
    return vertexai.generative_models.Image.from_bytes(img_byte_arr)

In [3]:
sneaker_image_paths = [
    "sneaker-collections/nike1.jpg",
    "sneaker-collections/nike2.jpg",
    "sneaker-collections/nike3.jpg",
    "sneaker-collections/nike4.jpg",
    "sneaker-collections/nike5.jpg",
    "sneaker-collections/adidas1.png",
    "sneaker-collections/adidas2.png",
    "sneaker-collections/adidas3.png",
    "sneaker-collections/adidas4.png",
    "sneaker-collections/adidas5.png"   
]
sneaker_images = [load_image_from_gcs(bucket_name, path) for path in sneaker_image_paths]

In [4]:
input_image_path = "user-input-images/input2.png"
input_image = load_image_from_gcs(bucket_name, input_image_path)

In [5]:
prompt = "Describe just the tshirt,shirt,trousers or jeans in the picture in brief"
contents = [
    prompt,
    input_image,
]
responses = multimodal_model.generate_content(contents, stream=True)

for response in responses:
    print(response.text)

 The picture shows a pair of blue jeans. The jeans are straight-legged and have a medium rise. They are made of a cotton blend fabric and have
 a zip fly and button closure.


In [8]:
user_input_prompt = f"""
        I am looking for adidas sneakers costing between 300 and 500 dirhams.
            """

instructions_prompt = f"""Fetch the colors and the product price first.Then only select 3 to 4 products 
                that match the user preferences.
                If no products match the user preference or budget, then say "Sorry, we do not have 
                products matching your preferences".
                Think step by step before responding.
                The User says : {user_input_prompt}
                """

output_prompt = """
            Output Format:
                1: Respond in json with 2 keys explanation and image_path.
                2: explanation must be your response to the above user request strictly matching their preferences
                and image_path must be the path of the product images that you have chosen. 
                3: Output Format must be strictly a json string in the below format with no illegal string terminations
                and must be inside quotes. For example : "{{'explanation':'','image_paths':''}}"
                Pay attention to the user preferences before responding.
                """

contents = [
    "Consider the following sneakers:",
    "Nike Air Max 1:",
    sneaker_images[0],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike1.jpg?authuser=4",
    "Nike Air max SC:",
    sneaker_images[1],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike2.jpg?authuser=4",
    "Nike Air Max 1 Black:",
    sneaker_images[2],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike3.jpg?authuser=4",
    "Nike Dunk Low Retro- White:",
    sneaker_images[3],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike4.jpg?authuser=4",
    "Nike Dunk Low Retro- Black:",
    sneaker_images[4],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike5.jpg?authuser=4",
    "Adidas-Sambda OG Shoes:",
    sneaker_images[5],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas1.png?authuser=4",
    "Adidas-Rivalry Hi shoes:",
    sneaker_images[6],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas2.png?authuser=4",
    "Adidas-Ultrabounce Shoes:",
    sneaker_images[7],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas3.png?authuser=4",
    "Adidas-Courtblock shoes:",
    sneaker_images[8],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas4.png?authuser=4",
    "Adidas-Stan Smith shoes:",
    sneaker_images[9],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas5.png?authuser=4",
    "Input Image:",
    input_image,
    instructions_prompt,
    output_prompt,
]

responses = multimodal_model.generate_content(contents, stream=True)

final_output = ""
for response in responses:
    llm_output= response.text
    clean_json_string = llm_output.strip()
    cleaned_output = clean_json_string.strip("`").replace("json\n", "", 1).strip()
    final_output += cleaned_output
    
print(final_output)

{
                "explanation": "Here are some adidas sneakers that match your preferences:",
                "image_paths": ["https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas1.png?authuser=4", "https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas2.png?authuser=4", "https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas3.png?authuser=4"]}


In [9]:
img_paths = json.loads(final_output)["image_paths"]

In [15]:
for i in img_paths:
    print(i)

https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas1.png?authuser=4
https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas2.png?authuser=4
https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas3.png?authuser=4


In [19]:
# pip install streamlit

import streamlit as st 
def display_image_from_gcs(image_path: str):
    """Download an image from Google Cloud Storage and display it."""
    try:
        client = storage.Client()
        bucket_name = "llm-demo-v1"
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(image_path)
        image_bytes = blob.download_as_bytes()

        # Display the image
        st.image(image_bytes, caption='Recommended Sneaker', use_column_width=True)

    except Exception as e:
        st.error(f"An error occurred while displaying the image from Google Cloud Storage: {str(e)}")

In [20]:
for i in img_paths:
    display_image_from_gcs(i)

2024-03-01 20:07:11.455 
  command:

    streamlit run /opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
